In [22]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import sys

sys.path.append('../utils')

from helpers import real_gdp_df

In [23]:
M2_df = pd.read_csv('data/M2SL.csv')

M2_df['DATE'] = pd.to_datetime(M2_df['DATE'])

M2_df.set_index('DATE', inplace=True)

# ---------------------------------------------- Frame
# M2_df = M2_df[M2_df.index >= pd.to_datetime('03-Oct-1980')]
# M2_df = M2_df[M2_df.index < pd.to_datetime('01-Apr-2015')]
# ----------------------------------------------

M2_df['change'] = M2_df['M2SL'] -  M2_df['M2SL'].shift(1)

M2_df['^ %'] = M2_df['M2SL'].pct_change()

# TODO think about it again
M2_df['Annualised ^ %'] = M2_df['^ %']

In [24]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=M2_df.index,
    y=M2_df['M2SL'],
    name='M2',
    fill='tonexty',
    fillcolor='rgba(0, 0, 255, 0.9)',
    line=dict(color='blue', width=1),
))

# ---------------------------------------------------------------------

Y = np.array(M2_df['M2SL'])
X = np.arange(1, len(Y) + 1)
m, c = np.polyfit(X, Y, 1)

# ---------------------------------------------------------------------

fig.add_trace(go.Scatter(
    x=M2_df.index, 
    y=m * X + c, 
    name='Trend (LR)',
    line=dict(color='red', width=1),
    showlegend=True
))

fig.update_layout(
    title=dict(text='M2'),
    width=1400,
    height=900,
    plot_bgcolor="black",
    paper_bgcolor="#212121",
    font=dict(color="white"),
    hovermode="x unified",
    yaxis=dict(title="Billions of Dollars", exponentformat="none", showgrid=False, zeroline=False),
    barmode="group",
    legend=dict(x=0.5, y=-0.2, orientation="h"),
    xaxis=dict(showgrid=False),
)

fig.show()

In [25]:
GDP_df = real_gdp_df()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=M2_df.index,
    y=M2_df['^ %']*100,
    name='M2',
    marker=dict(line=dict(color='rgba(0, 255, 0, .5)', width=2)),
    width=0.5,
))

filtered_gdp_df = GDP_df[GDP_df.index >= M2_df.index[0]]

fig.add_trace(go.Bar(
    x=filtered_gdp_df.index,
    y=filtered_gdp_df['GDP'],
    name='GDP',
    yaxis="y2",
    marker=dict(line=dict(color='rgba(255, 0, 0, 1)', width=2)),
    width=0.5,
))

# ------------------------------------------------------------------------
level_a = 0  # Level on left y-axis
level_b = 0  # Matching level on right y-axis

# Define the ranges for each axis
range_left = [M2_df['^ %'].min() * 100 - 1, M2_df['^ %'].max() * 100 + 1]  # Min and max for right y-axis (to be adjusted)
range_right = [filtered_gdp_df['GDP'].min() - 5, filtered_gdp_df['GDP'].max() + 5]  # Min and max for left y-axis

# Calculate scaling factor to align levels
proportion = (level_a - range_left[0]) / (range_left[1] - range_left[0])  # Position of level_a
range_right[1] = range_right[0] + (level_b - range_right[0]) / proportion  # Adjust right range
# ------------------------------------------------------------------------

fig.update_layout(
    title=dict(text='M2 Money Supply (Annualised Rate) vs GDP change'),
    plot_bgcolor="black",
    paper_bgcolor="#212121",
    font=dict(color="white"),
    width=1400,
    height=700,
    hovermode="x unified",
    yaxis=dict(title="M2 change %", exponentformat="none", range=range_left, showgrid=False, zeroline=False),
    yaxis2=dict(title="Real GDP in %", overlaying="y", side="right", range=range_right, showgrid=False, zeroline=False),
    barmode="group",
    legend=dict(x=0.5, y=-0.2, orientation="h"),
    xaxis=dict(showgrid=False),
)

fig.show()

- **Overview on M2 changes**

In [26]:
print(f"Average change is {M2_df['^ %'].mean() * 100}%")

Average change is 0.5486875470750534%


In [27]:
positive_filter = M2_df['^ %'] > 0
negative_filter = M2_df['^ %'] < 0
zero_filter = M2_df['^ %'] == 0

# count
total_items = len(M2_df)

# average change
avg_change = [
    M2_df[positive_filter]['^ %'].mean(),
    M2_df[negative_filter]['^ %'].mean(),
    0,
    ""
]

# frequency
frequency = [
    M2_df[positive_filter]['^ %'].count(),
    M2_df[negative_filter]['^ %'].count(),
    M2_df[zero_filter]['^ %'].count()
]
frequency.append(frequency[0]/frequency[1])

# frequency pct
frequency_pct = [(100 * frequency[i]/total_items) for i in range(3)]
frequency_pct.append("")

# Prob adj
prob_adj = [frequency_pct[i] * avg_change[i] for i in range(3)]
prob_adj.append("")

pd.DataFrame({
    "%": [100 * x for x in avg_change],
    "Frequency": frequency,
    "Frequency %": frequency_pct,
    "Prob Adjust % Change": prob_adj,
}, index=["Av Pos", "Av Neg", "Zero", "Ratio P/N"])

,%,Frequency,Frequency %,Prob Adjust % Change
Av Pos,0.596605,741.0000,93.797468,0.5596
Av Neg,-0.19104,48.0000,6.075949,-0.011607
Zero,0,0.0000,0.0,0.0
Ratio P/N,,15.4375,,


- **Stats**

In [28]:
from scipy.stats import describe

# Display the results

stats = describe(M2_df['^ %'].dropna().tolist())
pd.DataFrame(
    {
        'value': [
            str(stats.nobs),
            stats.minmax[0] * 100,
            stats.minmax[1] * 100,
            stats.mean * 100,
            M2_df['^ %'].median() * 100,
            M2_df['^ %'].mode(dropna=True)[0] * 100,
            stats.variance,
            stats.skewness,
            stats.kurtosis
        ]
    },
    index=['nobs', 'Min %', 'Max %', 'Mean %', "Median %", "Mode %", 'Variance', 'Skewness', 'Kurtosis'],
)

,value
nobs,789
Min %,-1.170114
Max %,6.372727
Mean %,0.548688
Median %,0.525647
Mode %,0.768049
Variance,0.000022
Skewness,4.119442
Kurtosis,43.525621


- Preview

In [29]:
# Define bins
bins = [0.5*i for i in range(-6, 16)]
bin_labels = [f"{bins[i]}% to {bins[i+1]}%" for i in range(len(bins) - 1)]
bin_labels[0] = f"Less than {bins[1]}%"
bin_labels[-1] = f"Greater than {bins[-2]}%"
# Assign data to bins
binned = pd.cut( M2_df['^ %'], bins=[i * 0.01 for i in bins], labels=bin_labels, include_lowest=True)

# Calculate frequency, probability, and cumulative probability
frequency = binned.value_counts().sort_index()
probability = 100 * frequency / frequency.sum()
cumulative_probability = probability.cumsum()

occurrence_frequencies = pd.DataFrame({
    'Frequency': frequency.values,
    'Probability %': probability.values,
    'Cumulative Probability %': cumulative_probability.values
}, index=bin_labels)

occurrence_frequencies

,Frequency,Probability %,Cumulative Probability %
Less than -2.5%,0,0.000000,0.000000
-2.5% to -2.0%,0,0.000000,0.000000
-2.0% to -1.5%,0,0.000000,0.000000
-1.5% to -1.0%,1,0.126743,0.126743
-1.0% to -0.5%,3,0.380228,0.506971
-0.5% to 0.0%,44,5.576679,6.083650
0.0% to 0.5%,316,40.050697,46.134347
0.5% to 1.0%,350,44.359949,90.494297
1.0% to 1.5%,62,7.858048,98.352345
1.5% to 2.0%,7,0.887199,99.239544


In [30]:
# plt.figure(figsize=(20, 10))

# plt.bar(occurrence_frequencies.index, occurrence_frequencies['Frequency'], color='skyblue', edgecolor='gray')

# # Add labels and title
# plt.title('M2 Annualised Rate %')
# plt.xlabel('')
# plt.ylabel('Frequency')
# plt.xticks(rotation=70) 
# # Show the chart
# plt.tight_layout()
# plt.show();

# --------------------------------------------------

fig = go.Figure()

fig.add_trace(go.Bar(
    x=occurrence_frequencies.index,
    y=occurrence_frequencies['Frequency'],
    name='M2 Annualised Rate',
    marker=dict(color='blue'),
))

fig.update_layout(
    title=dict(text='M2 Annualised Rate pct change %'),
    plot_bgcolor='rgb(230, 230,230)',
    width=700,
    height=700,
    yaxis=dict(title="Frequency", exponentformat="none", showgrid=False),
    barmode="group",
    xaxis=dict(showgrid=False),
    shapes=[
        dict(
            type="line",
            x0="0.0% to 0.5%",  
            x1="0.0% to 0.5%", 
            y0=0,  # Start of the line on the y-axis
            y1=350,  # End of the line on the y-axis
            line=dict(
                color="Red",
                width=2,
                dash="solid",
            ),
        )
    ]
)

fig.show()